<a href="https://colab.research.google.com/github/VeereshShringari/FreeML/blob/master/4_POSTagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Custom POS tagger

1. http://www.nltk.org/book/ch05.html
2. https://nlpforhackers.io/training-pos-tagger/

In [0]:
import nltk
from nltk import word_tokenize, pos_tag
#snowball tagger works for the 8 languages it is part of NLTK

In [2]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [0]:
tagged_sentences = nltk.corpus.treebank.tagged_sents()

In [4]:
len(tagged_sentences)


3914

In [11]:
tagged_sentences[5]

[('Lorillard', 'NNP'),
 ('Inc.', 'NNP'),
 (',', ','),
 ('the', 'DT'),
 ('unit', 'NN'),
 ('of', 'IN'),
 ('New', 'JJ'),
 ('York-based', 'JJ'),
 ('Loews', 'NNP'),
 ('Corp.', 'NNP'),
 ('that', 'WDT'),
 ('*T*-2', '-NONE-'),
 ('makes', 'VBZ'),
 ('Kent', 'NNP'),
 ('cigarettes', 'NNS'),
 (',', ','),
 ('stopped', 'VBD'),
 ('using', 'VBG'),
 ('crocidolite', 'NN'),
 ('in', 'IN'),
 ('its', 'PRP$'),
 ('Micronite', 'NN'),
 ('cigarette', 'NN'),
 ('filters', 'NNS'),
 ('in', 'IN'),
 ('1956', 'CD'),
 ('.', '.')]

### Custom Functions

In [0]:
# Feature Creation
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }


In [0]:
def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

### Train Test Split & Data Prep

In [8]:
# Split the dataset for training and testing
cutoff = int(.75 * len(tagged_sentences))
training_sentences = tagged_sentences[:cutoff]
test_sentences = tagged_sentences[cutoff:]
 
print(len(training_sentences))   # 2935
print(len(test_sentences))         # 979

2935
979


In [0]:
def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            X.append(features(untag(tagged), index))
            y.append(tagged[index][1])
    return X, y
 
X, y = transform_to_dataset(training_sentences)

In [14]:
X


[{'capitals_inside': False,
  'has_hyphen': False,
  'is_all_caps': False,
  'is_all_lower': False,
  'is_capitalized': True,
  'is_first': True,
  'is_last': False,
  'is_numeric': False,
  'next_word': 'Vinken',
  'prefix-1': 'P',
  'prefix-2': 'Pi',
  'prefix-3': 'Pie',
  'prev_word': '',
  'suffix-1': 'e',
  'suffix-2': 're',
  'suffix-3': 'rre',
  'word': 'Pierre'},
 {'capitals_inside': False,
  'has_hyphen': False,
  'is_all_caps': False,
  'is_all_lower': False,
  'is_capitalized': True,
  'is_first': False,
  'is_last': False,
  'is_numeric': False,
  'next_word': ',',
  'prefix-1': 'V',
  'prefix-2': 'Vi',
  'prefix-3': 'Vin',
  'prev_word': 'Pierre',
  'suffix-1': 'n',
  'suffix-2': 'en',
  'suffix-3': 'ken',
  'word': 'Vinken'},
 {'capitals_inside': False,
  'has_hyphen': False,
  'is_all_caps': True,
  'is_all_lower': True,
  'is_capitalized': True,
  'is_first': False,
  'is_last': False,
  'is_numeric': False,
  'next_word': '61',
  'prefix-1': ',',
  'prefix-2': ',',
  '

### Modeling

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
 # used for the one operation over the other out put of one  over the other
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy', random_state=2019))
])
 
clf.fit(X[:10000], y[:10000])   # Use only the first 10K samples if you're running it multiple times. It takes a fair bit :)
 
print('Training completed')

Training completed


In [15]:
X_test, y_test = transform_to_dataset(test_sentences)
print("Accuracy:", clf.score(X_test, y_test))

Accuracy: 0.8937007874015748


### Exercise

In [0]:
### Try changing the model and features and check for improvements

### NLTK Tagger

In [16]:
text = untag(tagged_sentences[0])
print(text)

['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.']


In [17]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [18]:
# Averaged perceptron tagger used by default
nltk.pos_tag(text)

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.')]

### Spacy Tagger

In [20]:
import spacy
import en_core_web_sm

nlp = en_core_web_sm.load()

doc = nlp(" ".join(text))

for token in doc:
    print(token.text, token.pos_)

Pierre PROPN
Vinken PROPN
, PUNCT
61 NUM
years NOUN
old ADJ
, PUNCT
will VERB
join VERB
the DET
board NOUN
as ADP
a DET
nonexecutive ADJ
director NOUN
Nov. PROPN
29 NUM
. PUNCT
